In [1]:
import sys
#from configure import auth_key
import requests
from time import sleep

def read_file(media_filename):
    with open(media_filename, 'rb') as _file:
        while True:
            data = _file.read(5242880)
            if not data:
                break
            yield data
            
def getAssemblyAI_Response(media_filename,word_boost=[]):
    try:
        #old auth key
        #auth_key = '1706479714ae4ebda977511e9e3a5495'
        #new auth key
        auth_key ='e6bd6d234190449290c9050a4509ee5e'
        # store global constants
        headers = {
           "authorization": auth_key,
           "content-type": "application/json"
        }
        transcript_endpoint = "https://api.assemblyai.com/v2/transcript"
        upload_endpoint = 'https://api.assemblyai.com/v2/upload'

        # upload our audio file
        upload_response = requests.post(
           upload_endpoint,
           headers=headers, data=read_file(media_filename)
        )
        print('Audio file uploaded')

        # send a request to transcribe the audio file
        transcript_request = {'audio_url': upload_response.json()['upload_url'],
                             #"word_boost": ['Talkdesk', 'AI', 'self-serve'],
                              'word_boost':word_boost,
                             "boost_param": "high",
                             "disfluencies": True,
                             "speaker_labels": True,
                             "iab_categories": True,
                             "sentiment_analysis": True,
                             "auto_chapters": True,
                             "entity_detection": True,
                             "auto_highlights": True
                            }
        transcript_response = requests.post(transcript_endpoint, json=transcript_request, headers=headers)
        print('Transcription Requested')
        # set up polling
        polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
        filename = transcript_response.json()['id'] + '.txt'
        # if our status isn’t complete, sleep and then poll again
        while polling_response.json()['status'] != 'completed' or polling_response.json()['status'] == 'error':
            sleep(30)
            polling_response = requests.get(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
            print("File is", polling_response.json()['status'])
            #if polling_response.json()['status'] == 'error':
               # break:
        requests.delete(transcript_endpoint+"/"+transcript_response.json()['id'], headers=headers)
        return polling_response
    except Exception as e:
        return []

In [2]:
audio_file_path = r'D:\Pavitra\NytroAI\Hiring_Usecase\Word_Testing.mp3'

In [3]:
polling_response = getAssemblyAI_Response(audio_file_path)

Audio file uploaded
Transcription Requested
File is completed


In [15]:
#output_json = polling_response.json()
#transcript = output_json['text']
#transcript

'Account Based Selling Account Executive Account Development Representative Bad Leads Bottom of the funnel Buyer Behavior Are you post or not.'